In [3]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

In [4]:
text_df = pd.read_csv("fake_or_real_news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

In [5]:
text_df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [6]:

partial_text = joined_text[:100000]

In [7]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [8]:
tokens

['daniel',
 'greenfield',
 'a',
 'shillman',
 'journalism',
 'fellow',
 'at',
 'the',
 'freedom',
 'center',
 'is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'c

In [9]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [10]:
unique_token_index

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '106': 5,
 '10th': 6,
 '11': 7,
 '12': 8,
 '12pm': 9,
 '13': 10,
 '14': 11,
 '15': 12,
 '16': 13,
 '160': 14,
 '17': 15,
 '187': 16,
 '19': 17,
 '1939': 18,
 '1960': 19,
 '197': 20,
 '1971': 21,
 '1972': 22,
 '1983': 23,
 '1984': 24,
 '1985': 25,
 '1990s': 26,
 '1992': 27,
 '1994': 28,
 '1996': 29,
 '1998': 30,
 '1999': 31,
 '2': 32,
 '20': 33,
 '200': 34,
 '2000': 35,
 '20001': 36,
 '2001': 37,
 '2002': 38,
 '2003': 39,
 '2004': 40,
 '2006': 41,
 '2007': 42,
 '2008': 43,
 '2009': 44,
 '2010': 45,
 '2011': 46,
 '2012': 47,
 '2013': 48,
 '2014': 49,
 '2015': 50,
 '2016': 51,
 '2020': 52,
 '21': 53,
 '2117': 54,
 '21st': 55,
 '21wire': 56,
 '22': 57,
 '227': 58,
 '23': 59,
 '24': 60,
 '241': 61,
 '27': 62,
 '2nd': 63,
 '3': 64,
 '30': 65,
 '300': 66,
 '31': 67,
 '33': 68,
 '3pm': 69,
 '4': 70,
 '40': 71,
 '400': 72,
 '42': 73,
 '49': 74,
 '5': 75,
 '50': 76,
 '500': 77,
 '51': 78,
 '52': 79,
 '57': 80,
 '5pm': 81,
 '6': 82,
 '60': 83,
 '

In [11]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [12]:
input_words

[['daniel',
  'greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center'],
 ['greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is'],
 ['a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a'],
 ['shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new'],
 ['journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york'],
 ['fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer'],
 ['at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing'],
 ['the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on'],
 ['freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on',
  'radical'],
 ['center',
  'is',
  'a',
  

In [13]:
next_word

['is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'comey',
 'is',
 'hillary',
 's',
 'opponent',
 'in',
 'this',
 'election',
 'the',
 'fbi',
 'is',
 'under',
 '

In [14]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [15]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [16]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

C:\Users\samed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.0511 - loss: 7.0634
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.0594 - loss: 6.6452
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.0660 - loss: 6.5159
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.0732 - loss: 6.3279
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.0878 - loss: 6.1017
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.1006 - loss: 5.8614
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.1202 - loss: 5.5830
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.1303 - loss: 5.3293
Epoch 9/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.1601 - loss: 5.0353
Epoch 10/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.1874 - loss: 4.7409


In [18]:
model.save('my_model.keras')

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model = load_model('my_model.keras')

In [20]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [34]:
possible = predict_next_word("The president of the United States announced that he", 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [35]:
print([unique_tokens[idx] for idx in possible])

['like', 'about', 'for', 'to', 'up', 'that', 'the', 'in', 'have', 'a']


In [36]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [39]:
generate_text("The president of the United States announced yesterday that he", 50, 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

'The president of the United States announced yesterday that he attack chorus theconversation carolina mattered retire considered defense president date for fbi one of the war for a house of us who s not one of the house and first us in new york in the us who are female representation in state of state women are more than the'